<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_Upside_Analysis_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [3]:
import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Stock_Analysis/data/Nifty100Scrips.csv')

mypf = mypf[mypf['InScope'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

#stock_ids

stock_ids = stock_n100

In [4]:
mypf = mypf[mypf.Forecast.notnull()]
stock_ids = mypf[mypf.Forecast.notnull()]['Symbol'].unique()

In [5]:
stock_ids.sort()
stock_ids

array(['ASIANPAINT', 'AXISBANK', 'BAJAJFINSV', 'BAJFINANCE', 'BANKBARODA',
       'BATAINDIA', 'BERGEPAINT', 'BHARTIARTL', 'BRITANNIA', 'CANBK',
       'CIPLA', 'COLPAL', 'DABUR', 'DEEPAKNTR', 'DIVISLAB', 'DMART',
       'DRREDDY', 'EICHERMOT', 'FEDERALBNK', 'GODREJCP', 'GODREJIND',
       'HAVELLS', 'HCLTECH', 'HDFC', 'HDFCBANK', 'HDFCLIFE', 'HEROMOTOCO',
       'HINDALCO', 'HINDUNILVR', 'IBREALEST', 'ICICIBANK', 'ICICIPRULI',
       'INDIANB', 'INDIGO', 'INDUSINDBK', 'INFY', 'IRCTC', 'ITC',
       'JUBLFOOD', 'KOTAKBANK', 'LT', 'LTIM', 'M&M', 'MARUTI',
       'MCDOWELL-N', 'MOTILALOFS', 'MUTHOOTFIN', 'NAM-INDIA',
       'NATIONALUM', 'NAUKRI', 'PFIZER', 'PIDILITIND', 'RELIANCE', 'SAIL',
       'SBILIFE', 'SBIN', 'SUNPHARMA', 'TATACONSUM', 'TATAMOTORS',
       'TATAPOWER', 'TATASTEEL', 'TCS', 'TECHM', 'TITAN', 'ULTRACEMCO',
       'VOLTAS', 'WHIRLPOOL', 'WIPRO', 'ZEEL'], dtype=object)

In [6]:
mypf['InPortfolio'].value_counts()

0    56
1    13
Name: InPortfolio, dtype: int64

In [7]:
mypf[mypf['InPortfolio'] == 1]['CCIFlag'].value_counts()

0    9
1    4
Name: CCIFlag, dtype: int64

In [8]:
# import necessary libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()
import pandas_ta as ta

In [9]:
import requests
from bs4 import BeautifulSoup

def get_current_pe(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  actual_link = link_blueprint + stock_name.upper()  + "/consolidated/"


  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")        
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:
    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
  except ValueError as ve:
    current_pe = 1000 

  return current_pe


In [10]:
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20 
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'
    
    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = yfin.Ticker(stock_symbol).history(period='2y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)     

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df['200_SMA_Lag10'] = stock_df['200_SMA'].shift(10)
    stock_df['Slope%_200'] = round((stock_df['200_SMA'] - stock_df['200_SMA_Lag10'])*100/stock_df['200_SMA'],2)
    stock_df['50_SMA_Lag5'] = stock_df['50_SMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_SMA'] - stock_df['50_SMA_Lag5'])*100/stock_df['50_SMA'],2)
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df['Min'] = round(min(stock_df['Close']),0)    
    #stock_df['Dev%_Max'] = round((stock_df['Close Price'] - stock_df['Max'])*100/stock_df['Max'],2)
    #stock_df['Dev%_Min'] = round((stock_df['Close Price'] - stock_df['Min'])*100/stock_df['Min'],2)
    stock_df['Premium%'] = round((stock_df['Close']-stock_df['Min'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Max']-stock_df['Min']),2)
    #stock_df.ta.macd(close='Close', fast=12, slow=26, append=True)
    #stock_df['Trend'] = 'Down'
    #stock_df.loc[stock_df['MACD_12_26_9'] - stock_df['MACDs_12_26_9'] > 10, 'Trend'] = 'Up'
    #stock_df.ta.eom(append=True)    
    stock_df.ta.rsi(append=True)
    #stock_df.rename(columns={'EOM_14_100000000':'EOM'}, inplace=True)
    stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    stock_df['Close'] = round(stock_df['Close'],0) 
    #stock_df['EOM'] = round(stock_df['EOM'],0) 
    #stock_df['EOM'] = '-ve'
    #stock_df.loc[stock_df['EOM_14_100000000'] > 0, 'EOM'] = '+ve'    
    stock_df['RSI'] = round(stock_df['RSI'],0) 
    stock_df.drop(['Open', 'Low', 'High', 'Volume','20_SMA','50_SMA','200_SMA','200_SMA_Lag10','50_SMA_Lag5'
                   #'MACD_12_26_9','MACDh_12_26_9','MACDs_12_26_9','EOM_14_100000000'
                   ], axis=1, inplace=True)   
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    current_pe = get_current_pe(stock_id)
    stock_id = stock_id.upper() + '.NS'    
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Forecast'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Forecast'].values[0]
    tmp['Med_PE'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['MedianPE'].values[0]
    tmp['Curr_PE'] = current_pe
    df_prec_dev = df_prec_dev.append(tmp)

In [11]:
stock_prec_dev('^NSEI').tail(5)

,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI
Date,,,,,,,,,,
2023-01-03 00:00:00+05:30,18233.0,^NSEI,5.73,0.49,0.37,18812.0,15294.0,83.53,16.47,49.0
2023-01-04 00:00:00+05:30,18043.0,^NSEI,4.59,0.48,0.32,18812.0,15294.0,78.14,21.86,43.0
2023-01-05 00:00:00+05:30,17992.0,^NSEI,4.28,0.45,0.27,18812.0,15294.0,76.70,23.30,41.0
2023-01-06 00:00:00+05:30,17859.0,^NSEI,3.49,0.42,0.23,18812.0,15294.0,72.92,27.08,38.0
2023-01-09 00:00:00+05:30,18056.0,^NSEI,4.61,0.38,0.19,18812.0,15294.0,78.52,21.48,45.0


In [12]:
# Upside% Calculations
df_prec_dev['Upside%FC'] = round((df_prec_dev['Forecast']-df_prec_dev['Close'])*100/(df_prec_dev['Close']),0)
df_prec_dev['Upside%52W'] = round((df_prec_dev['Max']-df_prec_dev['Close'])*100/(df_prec_dev['Close']),0)
#df_prec_dev['Upside%PE'] = round((df_prec_dev['Med_PE']-df_prec_dev['Curr_PE'])*100/(df_prec_dev['Curr_PE']),0)
df_prec_dev = df_prec_dev.sort_values(by = 'Upside%FC', ascending=False)
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)
#df_prec_dev[df_prec_dev['Trend'] == 'Up'].sort_values(by = 'Upside%', ascending=False)
df_prec_dev.drop(['Date'], axis=1, inplace=True)
cols = ['Stock', 'Close', 'Dev%_200', 'Slope%_200', 'Slope%_50', 'Max', 'Min',
       'Premium%', 'Discount%', 'RSI', 'Forecast', 'Med_PE', 'Curr_PE',
       'Upside%FC', 'Upside%52W']
df_prec_dev = df_prec_dev[cols]

In [13]:
# portfolio stocks
mypf['Stock'] = mypf['Symbol'] + '.NS'
tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%52W', ascending=False)

qualified stocks: 13


,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W
35,TECHM.NS,1014.0,-6.63,-1.93,-0.29,1684.0,935.0,10.55,89.45,46.0,1131.0,18.7,18.60,12.0,66.0
19,VOLTAS.NS,816.0,-16.16,-2.06,-0.72,1315.0,778.0,7.04,92.96,49.0,974.0,43.4,153.00,19.0,61.0
7,NAUKRI.NS,3651.0,-9.61,-0.87,-0.20,5667.0,3363.0,12.49,87.51,29.0,4677.0,78.8,15.90,28.0,55.0
32,MUTHOOTFIN.NS,1077.0,-2.26,-1.18,0.37,1516.0,956.0,21.62,78.38,49.0,1217.0,12.5,11.50,13.0,41.0
1,BERGEPAINT.NS,567.0,-9.63,-0.80,-0.17,785.0,558.0,3.81,96.19,30.0,782.0,72.9,58.20,38.0,38.0
4,MOTILALOFS.NS,711.0,-7.04,-0.92,0.15,960.0,654.0,18.68,81.32,58.0,930.0,19.7,9.78,31.0,35.0
10,TATAMOTORS.NS,389.0,-8.97,-0.23,-0.24,525.0,372.0,10.92,89.08,41.0,491.0,19.2,1000.00,26.0,35.0
56,DIVISLAB.NS,3464.0,-7.06,-1.21,-0.44,4595.0,3266.0,14.89,85.11,55.0,3560.0,45.2,30.80,3.0,33.0
2,ICICIPRULI.NS,465.0,-9.86,0.00,-0.85,615.0,440.0,14.34,85.66,53.0,636.0,60.9,78.10,37.0,32.0
21,INFY.NS,1489.0,-2.13,-0.99,-0.13,1898.0,1351.0,25.16,74.84,42.0,1733.0,23.0,27.20,16.0,27.0


In [14]:
# CCI buy
tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['CCIFlag'] == 1].Stock.values)]
tmp_df = tmp_df[tmp_df['Upside%52W'] > 30]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%52W', ascending=False)

qualified stocks: 6


,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W
43,WIPRO.NS,391.0,-10.39,-2.29,0.25,687.0,375.0,5.03,94.97,49.0,425.0,19.0,19.0,9.0,76.0
35,TECHM.NS,1014.0,-6.63,-1.93,-0.29,1684.0,935.0,10.55,89.45,46.0,1131.0,18.7,18.6,12.0,66.0
44,LTIM.NS,4278.0,-7.71,-1.98,-0.95,7121.0,3796.0,14.51,85.49,41.0,4675.0,26.6,49.4,9.0,66.0
8,JUBLFOOD.NS,499.0,-10.30,-0.18,-1.66,785.0,463.0,11.10,88.90,34.0,636.0,87.9,69.3,27.0,57.0
1,BERGEPAINT.NS,567.0,-9.63,-0.80,-0.17,785.0,558.0,3.81,96.19,30.0,782.0,72.9,58.2,38.0,38.0
56,DIVISLAB.NS,3464.0,-7.06,-1.21,-0.44,4595.0,3266.0,14.89,85.11,55.0,3560.0,45.2,30.8,3.0,33.0


In [15]:
tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df['Upside%52W']*100/tmp_df['Upside%52W'].sum()

1      8.242950
2      6.941432
4      7.592191
7     11.930586
10     7.592191
11     5.206074
13     0.650759
19    13.232104
21     5.856833
32     8.893709
34     2.386117
35    14.316703
56     7.158351
Name: Upside%52W, dtype: float64

In [16]:
# heavy upside
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[tmp_df['Upside%52W'] > 30]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%52W', ascending=False)

qualified stocks: 13


,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W
14,IBREALEST.NS,77.0,-3.63,-1.25,0.00,174.0,60.0,15.00,85.00,40.0,95.0,11.3,1000.00,23.0,126.0
43,WIPRO.NS,391.0,-10.39,-2.29,0.25,687.0,375.0,5.03,94.97,49.0,425.0,19.0,19.00,9.0,76.0
44,LTIM.NS,4278.0,-7.71,-1.98,-0.95,7121.0,3796.0,14.51,85.49,41.0,4675.0,26.6,49.40,9.0,66.0
8,JUBLFOOD.NS,499.0,-10.30,-0.18,-1.66,785.0,463.0,11.10,88.90,34.0,636.0,87.9,69.30,27.0,57.0
46,NATIONALUM.NS,83.0,-0.12,-2.41,1.30,128.0,67.0,26.07,73.93,61.0,90.0,10.3,6.02,8.0,54.0
0,GODREJIND.NS,449.0,-0.94,-0.66,0.00,644.0,405.0,18.31,81.69,54.0,638.0,39.5,19.60,42.0,43.0
5,NAM-INDIA.NS,249.0,-10.88,-1.08,-0.77,355.0,245.0,3.32,96.68,37.0,322.0,29.9,23.10,29.0,43.0
68,IRCTC.NS,638.0,-7.45,-0.73,-1.42,899.0,570.0,20.58,79.42,36.0,571.0,NaN,1000.00,-11.0,41.0
30,TATAPOWER.NS,206.0,-9.34,-0.88,-0.91,287.0,195.0,11.74,88.26,40.0,232.0,20.4,25.90,13.0,39.0
6,ZEEL.NS,237.0,-5.56,0.00,-1.18,325.0,206.0,26.09,73.91,36.0,303.0,24.6,28.10,28.0,37.0


In [17]:
# near 52W low
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[tmp_df['Discount%'] > 87]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Premium%', ascending=True)

qualified stocks: 4


,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W
5,NAM-INDIA.NS,249.0,-10.88,-1.08,-0.77,355.0,245.0,3.32,96.68,37.0,322.0,29.9,23.1,29.0,43.0
43,WIPRO.NS,391.0,-10.39,-2.29,0.25,687.0,375.0,5.03,94.97,49.0,425.0,19.0,19.0,9.0,76.0
8,JUBLFOOD.NS,499.0,-10.30,-0.18,-1.66,785.0,463.0,11.10,88.90,34.0,636.0,87.9,69.3,27.0,57.0
30,TATAPOWER.NS,206.0,-9.34,-0.88,-0.91,287.0,195.0,11.74,88.26,40.0,232.0,20.4,25.9,13.0,39.0


In [18]:
# near 52W high
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[tmp_df['Discount%'] < 10]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=True)

qualified stocks: 9


,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W
66,INDIANB.NS,299.0,50.91,3.54,2.16,300.0,127.0,99.31,0.69,59.0,284.0,7.4,8.51,-5.0,0.0
59,TATASTEEL.NS,117.0,31.97,3.37,0.92,119.0,44.0,97.93,2.07,61.0,118.0,6.0,5.11,1.0,2.0
39,AXISBANK.NS,954.0,23.29,1.55,0.55,962.0,625.0,97.70,2.30,61.0,1060.0,31.3,16.00,11.0,1.0
31,M&M.NS,1310.0,15.21,2.20,-0.16,1354.0,684.0,93.42,6.58,61.0,1475.0,20.5,19.40,13.0,3.0
29,FEDERALBNK.NS,135.0,21.53,1.80,0.00,139.0,82.0,92.81,7.19,51.0,152.0,11.0,11.50,13.0,3.0
62,CANBK.NS,325.0,33.28,2.46,1.27,337.0,175.0,92.72,7.28,54.0,322.0,8.8,6.32,-1.0,4.0
61,BANKBARODA.NS,185.0,42.46,3.85,2.31,193.0,89.0,92.50,7.50,58.0,183.0,13.5,9.72,-1.0,4.0
65,LT.NS,2123.0,16.37,1.04,0.72,2195.0,1453.0,90.24,9.76,55.0,2036.0,NaN,31.00,-4.0,3.0
22,INDUSINDBK.NS,1215.0,16.60,1.54,0.59,1264.0,771.0,90.06,9.94,52.0,1402.0,22.8,15.50,15.0,4.0


In [19]:
# turning around (exhaustive)
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] > -.5) & (tmp_df['Slope%_200'] < 0)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 7


,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W
58,TCS.NS,3313.0,0.76,-0.49,0.39,3965.0,2974.0,34.22,65.78,53.0,3385.0,28.1,30.6,2.0,20.0
12,WHIRLPOOL.NS,1498.0,-7.16,-0.31,-0.53,1871.0,1413.0,18.46,81.54,46.0,1854.0,72.2,73.3,24.0,25.0
40,DMART.NS,3870.0,-4.00,-0.20,-0.84,4634.0,3231.0,45.54,54.46,41.0,4282.0,138.7,109.0,11.0,20.0
18,BAJAJFINSV.NS,1413.0,-6.76,-0.20,-1.11,1836.0,1093.0,43.01,56.99,27.0,1691.0,39.2,41.2,20.0,30.0
3,BAJFINANCE.NS,5985.0,-10.19,-0.21,-1.27,7824.0,5265.0,28.14,71.86,25.0,7966.0,50.8,36.5,33.0,31.0
23,DEEPAKNTR.NS,1932.0,-6.05,-0.39,-1.36,2650.0,1713.0,23.33,76.67,33.0,2231.0,28.4,28.7,15.0,37.0
8,JUBLFOOD.NS,499.0,-10.30,-0.18,-1.66,785.0,463.0,11.10,88.90,34.0,636.0,87.9,69.3,27.0,57.0


In [20]:
# turning around (compact)
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] < 0) & (tmp_df['Slope%_50'] > 0)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%52W', ascending=False)

qualified stocks: 6


,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W
43,WIPRO.NS,391.0,-10.39,-2.29,0.25,687.0,375.0,5.03,94.97,49.0,425.0,19.0,19.00,9.0,76.0
46,NATIONALUM.NS,83.0,-0.12,-2.41,1.30,128.0,67.0,26.07,73.93,61.0,90.0,10.3,6.02,8.0,54.0
42,HINDALCO.NS,469.0,9.15,-1.16,1.56,626.0,314.0,49.79,50.21,54.0,514.0,10.4,7.56,10.0,33.0
63,SAIL.NS,87.0,7.96,-1.23,1.19,108.0,63.0,54.33,45.67,58.0,85.0,6.9,7.58,-2.0,24.0
53,HCLTECH.NS,1066.0,6.09,-0.50,0.19,1291.0,865.0,47.23,52.77,56.0,1117.0,16.3,20.80,5.0,21.0
58,TCS.NS,3313.0,0.76,-0.49,0.39,3965.0,2974.0,34.22,65.78,53.0,3385.0,28.1,30.60,2.0,20.0


In [21]:
# bull run
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] > 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 5


,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W
47,INDIGO.NS,2009.0,8.50,0.59,1.36,2265.0,1549.0,64.31,35.69,53.0,2176.0,30.7,1000.0,8.0,13.0
26,HDFCBANK.NS,1594.0,9.36,0.75,1.00,1662.0,1281.0,82.27,17.73,43.0,1813.0,26.0,21.4,14.0,4.0
28,HDFC.NS,2606.0,9.37,0.84,0.91,2729.0,2053.0,81.86,18.14,43.0,2940.0,23.0,19.7,13.0,5.0
50,ULTRACEMCO.NS,7034.0,9.53,0.75,0.86,7825.0,5148.0,70.45,29.55,52.0,7420.0,33.7,30.5,5.0,11.0
25,DABUR.NS,548.0,0.29,0.18,0.35,604.0,480.0,54.52,45.48,37.0,625.0,53.6,54.2,14.0,10.0


In [22]:
# about to turn around
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] < 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 10


,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W
42,HINDALCO.NS,469.0,9.15,-1.16,1.56,626.0,314.0,49.79,50.21,54.0,514.0,10.4,7.56,10.0,33.0
46,NATIONALUM.NS,83.0,-0.12,-2.41,1.30,128.0,67.0,26.07,73.93,61.0,90.0,10.3,6.02,8.0,54.0
63,SAIL.NS,87.0,7.96,-1.23,1.19,108.0,63.0,54.33,45.67,58.0,85.0,6.9,7.58,-2.0,24.0
58,TCS.NS,3313.0,0.76,-0.49,0.39,3965.0,2974.0,34.22,65.78,53.0,3385.0,28.1,30.60,2.0,20.0
43,WIPRO.NS,391.0,-10.39,-2.29,0.25,687.0,375.0,5.03,94.97,49.0,425.0,19.0,19.00,9.0,76.0
53,HCLTECH.NS,1066.0,6.09,-0.50,0.19,1291.0,865.0,47.23,52.77,56.0,1117.0,16.3,20.80,5.0,21.0
0,GODREJIND.NS,449.0,-0.94,-0.66,0.00,644.0,405.0,18.31,81.69,54.0,638.0,39.5,19.60,42.0,43.0
14,IBREALEST.NS,77.0,-3.63,-1.25,0.00,174.0,60.0,15.00,85.00,40.0,95.0,11.3,1000.00,23.0,126.0
5,NAM-INDIA.NS,249.0,-10.88,-1.08,-0.77,355.0,245.0,3.32,96.68,37.0,322.0,29.9,23.10,29.0,43.0
44,LTIM.NS,4278.0,-7.71,-1.98,-0.95,7121.0,3796.0,14.51,85.49,41.0,4675.0,26.6,49.40,9.0,66.0


In [23]:
# star stocks performance

star_stocks = ['HINDUNILVR.NS', 'BAJFINANCE.NS', 'EICHERMOT.NS', 'FEDERALBNK.NS',
          'CANBK.NS', 'TCS.NS', 'M&M.NS', 'LTIM.NS', 'ICICIBANK.NS',
          'AXISBANK.NS', 'HCLTECH.NS', 'TECHM.NS', 'WIPRO.NS',
          'INDUSINDBK.NS', 'JUBLFOOD.NS', 'DMART.NS', 'DABUR.NS',
          'TATAPOWER.NS', 'BHARTIARTL.NS']

df_prec_dev[df_prec_dev['Stock'].isin(star_stocks)].sort_values(by = 'Premium%', ascending=True)

,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W
43,WIPRO.NS,391.0,-10.39,-2.29,0.25,687.0,375.0,5.03,94.97,49.0,425.0,19.0,19.00,9.0,76.0
35,TECHM.NS,1014.0,-6.63,-1.93,-0.29,1684.0,935.0,10.55,89.45,46.0,1131.0,18.7,18.60,12.0,66.0
8,JUBLFOOD.NS,499.0,-10.30,-0.18,-1.66,785.0,463.0,11.10,88.90,34.0,636.0,87.9,69.30,27.0,57.0
30,TATAPOWER.NS,206.0,-9.34,-0.88,-0.91,287.0,195.0,11.74,88.26,40.0,232.0,20.4,25.90,13.0,39.0
44,LTIM.NS,4278.0,-7.71,-1.98,-0.95,7121.0,3796.0,14.51,85.49,41.0,4675.0,26.6,49.40,9.0,66.0
3,BAJFINANCE.NS,5985.0,-10.19,-0.21,-1.27,7824.0,5265.0,28.14,71.86,25.0,7966.0,50.8,36.50,33.0,31.0
58,TCS.NS,3313.0,0.76,-0.49,0.39,3965.0,2974.0,34.22,65.78,53.0,3385.0,28.1,30.60,2.0,20.0
40,DMART.NS,3870.0,-4.00,-0.20,-0.84,4634.0,3231.0,45.54,54.46,41.0,4282.0,138.7,109.00,11.0,20.0
53,HCLTECH.NS,1066.0,6.09,-0.50,0.19,1291.0,865.0,47.23,52.77,56.0,1117.0,16.3,20.80,5.0,21.0
25,DABUR.NS,548.0,0.29,0.18,0.35,604.0,480.0,54.52,45.48,37.0,625.0,53.6,54.20,14.0,10.0
